In [1]:
import numpy as np
import pandas as pd
import re
import time
import os.path
import shutil

path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\processed\\'
generated_path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\generated\\'

def strip(df):
    df['Memo']=df['Memo'].str.strip()
    df['Subcategory']=df['Subcategory'].str.strip()
    if 'Description' in df.columns:
        df['Description']=df['Description'].str.strip()
    return df

def backup_file(filename):
    timestr = time.strftime('%Y%m%d-%H%M%S')
    if os.path.isfile(filename):
        shutil.copy2(filename,filename+'.bak_' + timestr)
    return filename

def load_bank_worksheet(spreadsheet, worksheet):
    input_file = path + spreadsheet + '.xlsm'
    df = pd.read_excel(input_file,sheet_name=worksheet)
    df.drop('Import', axis=1, inplace=True)
    df.drop('Number', axis=1, inplace=True)
    df = df.astype({"Description": str})
    df = df[~df.Account.isnull()]
    return df

def load_beals_worksheet(spreadsheet, worksheet):
    input_file = path + spreadsheet + '.xlsm'
    df = pd.read_excel(input_file,sheet_name=worksheet)
    return df

def extract_transactions():
    bealsDir=generated_path + 'beals';
    if not os.path.exists(bealsDir):
        os.mkdir(bealsDir)

    dfAll = pd.DataFrame(columns=['Date','Account','Amount','Subcategory','Memo','Property','Description'])
    for spreadsheet in spreadsheets:
        print('Processing: ' + spreadsheet)
        dfMt=load_bank_worksheet(spreadsheet, 'MT')
        dfIv=load_bank_worksheet(spreadsheet, 'IV')
        df40=load_bank_worksheet(spreadsheet, 'IV (4040)')
        dfAll=pd.concat([dfMt,dfIv,df40]).sort_index()
        dfAll.loc[(dfAll.Description=='nan'),'Description']=''
        dfAll['Property'] = dfAll['Property'].fillna('')
        dfAll=strip(dfAll)
        dfAll=dfAll.loc[:, ~dfAll.columns.str.contains('^Unnamed')]
        dfAll=dfAll.set_index('Date')

        # Save file
        dateStr = re.search(r"^Beals_RentalStatement_([A-Z]{3}\d{4})_.*$", spreadsheet)
        #file=backup_file(generated_path + dateStr.group(1) + '_coded.csv')
        file=backup_file(generated_path + dateStr.group(1) + '.csv')
        dfAll.to_csv(file)
        
        # Extract Beals
        dfBeals=load_beals_worksheet(spreadsheet, 'Beals')
        file=backup_file(bealsDir + '\\Beals_' + dateStr.group(1) + '.csv')
        dfBeals.to_csv(file)

In [2]:
spreadsheets=[ \
              #'Beals_RentalStatement_DEC2016_20032017_1799', \
              'Beals_RentalStatement_JAN2017_20032017_1803', 'Beals_RentalStatement_FEB2017_06042017_1816', \
              'Beals_RentalStatement_MAR2017_06042017_1817', 'Beals_RentalStatement_APR2017_15062017_1868', \
              'Beals_RentalStatement_MAY2017_15062017_1872',  \
              'Beals_RentalStatement_JUN2017_03112017_1902', 'Beals_RentalStatement_JUL2017_17082017_1894', \
              'Beals_RentalStatement_AUG2017_03112017_1904', 'Beals_RentalStatement_SEP2017_03112017_1917', \
              'Beals_RentalStatement_OCT2017_03112017_1918', 'Beals_RentalStatement_NOV2017_20122017_1922', \
              'Beals_RentalStatement_DEC2017_16012018_1924', 'Beals_RentalStatement_JAN2018_07032018_1928', \
              'Beals_RentalStatement_FEB2018_07032018_1933', 'Beals_RentalStatement_MAR2018_08052018_1953', \
              'Beals_RentalStatement_APR2018_08052018_1954', 'Beals_RentalStatement_MAY2018_11072018_1959', \
              
              'Beals_RentalStatement_JUN2018_11072018_1961', 'Beals_RentalStatement_JUL2018_21082018_1964', \
              'Beals_RentalStatement_AUG2018_15092018_1968', 'Beals_RentalStatement_SEP2018_20102018_1971', \
              'Beals_RentalStatement_OCT2018_16112018_1978', 'Beals_RentalStatement_NOV2018_22122018_1979', \
              'Beals_RentalStatement_DEC2018_18012019_1986', 'Beals_RentalStatement_JAN2019_23022019_1988', \

              'Beals_RentalStatement_FEB2019_19042019_1991', 'Beals_RentalStatement_MAR2019_19042019_1995', \
              'Beals_RentalStatement_APR2019_30062019_2036', 'Beals_RentalStatement_MAY2019_30062019_2039', \
              'Beals_RentalStatement_JUN2019_08082019_2041', 'Beals_RentalStatement_JUL2019_12102019_2048', \
              'Beals_RentalStatement_AUG2019_12102019_2049', 'Beals_RentalStatement_SEP2019_01122019_2054', \
              'Beals_RentalStatement_OCT2019_05042020_2058', 'Beals_RentalStatement_NOV2019_05042020_2059', \
              'Beals_RentalStatement_DEC2019_05042020_2061', 'Beals_RentalStatement_JAN2020_05042020_2063', \

              'Beals_RentalStatement_FEB2020_05042020_2065', 'Beals_RentalStatement_MAR2020_20082020_2066', \
              'Beals_RentalStatement_APR2020_21082020_2071', 'Beals_RentalStatement_MAY2020_28082020_2082', \
              'Beals_RentalStatement_JUN2020_28082020_2083', 'Beals_RentalStatement_JUL2020_28082020_2084', \
              'Beals_RentalStatement_AUG2020_23062021_2098', 'Beals_RentalStatement_SEP2020_23062021_2105', \
              'Beals_RentalStatement_OCT2020_23062021_2109', 'Beals_RentalStatement_NOV2020_23062021_2124', \
              'Beals_RentalStatement_DEC2020_23062021_2125', 'Beals_RentalStatement_JAN2021_30062021_2138', \
             
              'Beals_RentalStatement_FEB2021_30062021_2139', 'Beals_RentalStatement_MAR2021_30062021_2141', \
              'Beals_RentalStatement_APR2021_30062021_2142', 'Beals_RentalStatement_MAY2021_02072021_2144', \
              'Beals_RentalStatement_JUN2021_02102021_2154', 'Beals_RentalStatement_JUL2021_02102021_2159', \
              'Beals_RentalStatement_AUG2021_02102021_2160', 'Beals_RentalStatement_SEP2021_02102021_2161', \
              'Beals_RentalStatement_OCT2021_21062022_2167', 'Beals_RentalStatement_NOV2021_21062022_2168', \
              'Beals_RentalStatement_DEC2021_21062022_2169', 'Beals_RentalStatement_JAN2022_21062022_2170', \
              'Beals_RentalStatement_FEB2022_21062022_2171', 'Beals_RentalStatement_MAR2022_21062022_2172', \
              'Beals_RentalStatement_APR2022_21062022_2173', 'Beals_RentalStatement_MAY2022_21062022_2174', \
              'Beals_RentalStatement_JUN2022_11092022_2183', 'Beals_RentalStatement_JUL2022_11092022_2184' \
             ]

### Extract bank and Beals data from Rent Statements

- the output files 'MMMYYYY_coded.csv' (because the data already has property codes)

In [3]:
extract_transactions()

Processing: Beals_RentalStatement_JAN2017_20032017_1803
Processing: Beals_RentalStatement_FEB2017_06042017_1816
Processing: Beals_RentalStatement_MAR2017_06042017_1817
Processing: Beals_RentalStatement_APR2017_15062017_1868
Processing: Beals_RentalStatement_MAY2017_15062017_1872
Processing: Beals_RentalStatement_JUN2017_03112017_1902
Processing: Beals_RentalStatement_JUL2017_17082017_1894
Processing: Beals_RentalStatement_AUG2017_03112017_1904
Processing: Beals_RentalStatement_SEP2017_03112017_1917
Processing: Beals_RentalStatement_OCT2017_03112017_1918
Processing: Beals_RentalStatement_NOV2017_20122017_1922


C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Processing: Beals_RentalStatement_DEC2017_16012018_1924
Processing: Beals_RentalStatement_JAN2018_07032018_1928
Processing: Beals_RentalStatement_FEB2018_07032018_1933
Processing: Beals_RentalStatement_MAR2018_08052018_1953
Processing: Beals_RentalStatement_APR2018_08052018_1954
Processing: Beals_RentalStatement_MAY2018_11072018_1959
Processing: Beals_RentalStatement_JUN2018_11072018_1961
Processing: Beals_RentalStatement_JUL2018_21082018_1964
Processing: Beals_RentalStatement_AUG2018_15092018_1968
Processing: Beals_RentalStatement_SEP2018_20102018_1971
Processing: Beals_RentalStatement_OCT2018_16112018_1978
Processing: Beals_RentalStatement_NOV2018_22122018_1979
Processing: Beals_RentalStatement_DEC2018_18012019_1986
Processing: Beals_RentalStatement_JAN2019_23022019_1988
Processing: Beals_RentalStatement_FEB2019_19042019_1991
Processing: Beals_RentalStatement_MAR2019_19042019_1995
Processing: Beals_RentalStatement_APR2019_30062019_2036
Processing: Beals_RentalStatement_MAY2019_300620